# Erhebung und Analyse von Sensordaten - Time of flight Kamera


## Step 0: Development environment

The following lines will enable the autoreload extension for this Jupyter Notebook. This provides the functionality to automatically reload modules upon editing those, without the need to restart the entire kernel, thus making it necessary to re-run all cells before.

In [ ]:
%load_ext autoreload

# '2' means that all modules will be reloaded automatically before executing any cell
%autoreload 2

## Step 1: Load data
Get the input data sets inside 'Messdaten' as a dictionary with the following structure:
```json
{
    'Input_path_1': np.array,
    'Input_path_2': np.array,
    // ...
    
}
```
Where `np.array` is a three dimensional array containing depth data frames of size 80 * 60 

In [ ]:

import utilities


input_file_paths = utilities.load_input_file_paths()
data_sets = utilities.load_depth_data_sets(input_file_paths)


## Step 2: Visualize the data

Save the loaded data frames as gif files to `output/<input-file-name>/raw.gif` and display them in this notebook.

In [ ]:
from IPython.display import Image, Markdown, display

display(Markdown('### Heatmap animations created for input data:'))

for file_name, frames in data_sets.items():
    try:
        output_path = utilities.create_gif_from_frames(frames, f'output/{file_name}/raw.gif')
    except ValueError as e:
        print(e)
        continue

    display(Image(filename=output_path, width=200), Markdown(f'*{file_name}*'))


## Step 3: Manipulate the data

1. Create a background map for each frame by calculating the mean depth map per sequence. 
2. Use the background map to subtract it from each frame in the sequence
3. Create a foreground mask:  
If the remaining depth value of a pixel is still bigger than a certain threshold, the pixel is considered foreground (value 1)
Otherwise the pixel is part of the background (value 0)

> The foreground masks are saved in a data structure similar to the original frame data

In [ ]:
import filters
import numpy as np

display(Markdown('### Foreground masks created for input data:'))

foreground_masks = {}

for file_name, frames in data_sets.items():
    background = filters.compute_background(frames)

    foreground_mask = [filters.subtract_background_from_frame(frame, background, 1000) for frame in frames]
    foreground_mask = np.stack(foreground_mask)
    
    foreground_masks[file_name] = foreground_mask
    
    try:
        output_path = utilities.create_gif_from_frames(foreground_mask, f'output/{file_name}/foreground_mask.gif')

        display(Image(filename=output_path, width=200))
        display(Markdown(f'*{file_name}*'))
    except ValueError as e:
        print(e)
        continue
    

## Step 4: Analyze the data

In [ ]:
import ipywidgets as widgets

slider = widgets.IntSlider(
    value=50,
    min=1,
    max=100,
    step=1,
    description='Person size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
display(Markdown('Specify the amount of pixels, a person should be represented by in the image:'))
display(slider)


Use cv2 to count shapes in each frame and count the shape as person if it has a certain size. The resulting array of detected people per frame is saved to a similar data structure as the input data (dictionary with file names as keys):

In [ ]:
import vision

person_size = slider.value

display(Markdown(f'*Counting people per sequence and calculating movement data...*'))

people_counts = {}
movement_data = {}

for file_name, foreground_mask in foreground_masks.items():
    # collect the amount of people per frame
    people_counts[file_name] = [vision.count_people(mask, person_size=person_size) for mask in foreground_mask]

    # collect movement data
    movements_in_data_set = []
    
    for index in range(len(foreground_mask) - 1):
        movements_in_data_set.append(vision.count_and_track_people_direction(foreground_mask[index], foreground_mask[index + 1]))

    movement_data[file_name] = movements_in_data_set
    
    print(f'Max people in frame ({file_name}):', max(people_counts[file_name]))

Label each frame with the amount of people detected and mark center points for each detected person:

In [ ]:
for file_name in data_sets:
    frame_texts = []
    frame_points = []

    for movement in movement_data[file_name]:
        up = 0
        down = 0
        curr_points = []

        for person in movement:
            if person[2] == 'up':
                up += 1
            elif person[2] == 'down':
                down += 1
            person_x, person_y = person[0]
            curr_points.append([person_x, person_y])

        frame_points.append(curr_points)

        frame_text = f'People: {len(movement)} Up: {up} Down: {down}'
        frame_texts.append(frame_text)

    utilities.add_text_to_gif(f'output/{file_name}/raw.gif', frame_texts)
    utilities.add_points_to_gif(f'output/{file_name}/raw.gif', frame_points)